In [1]:
# 📌 Cell 2 — Set Up Folder Structure
import os
folders = [
    'data/ai/cars',
    'data/ai/buildings',
    'data/ai/medical',
    'data/_extra'
]
for folder in folders:
    os.makedirs(folder, exist_ok=True)
print('✅ AI folders ready')

✅ AI folders ready


In [2]:
# 📌 Install required packages for SDXL
%pip install -q diffusers==0.30.2 transformers==4.44.2 accelerate==0.34.2 safetensors==0.4.4
%pip install --index-url https://download.pytorch.org/whl/cpu torch==2.3.0+cpu

Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://download.pytorch.org/whl/cpu
Looking in indexes: https://download.pytorch.org/whl/cpu
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import torch
import diffusers
import transformers
print("torch:", torch.__version__)
print("diffusers:", diffusers.__version__)
print("transformers:", transformers.__version__)

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


torch: 2.3.0+cpu
diffusers: 0.30.2
transformers: 4.44.2


In [ ]:
from diffusers import StableDiffusionXLPipeline
import torch
from PIL import Image
from tqdm import tqdm
import os
import random

def generate_topic(topic_name, prompt_list, out_dir, n_images):
    os.makedirs(out_dir, exist_ok=True)
    try:
        pipe = StableDiffusionXLPipeline.from_pretrained(
            "stabilityai/stable-diffusion-xl-base-1.0",
            torch_dtype=torch.float32,
            use_safetensors=True
        )
        pipe.to("cpu")
    except Exception as e:
        print(f"Error loading SDXL pipeline for {topic_name}: {e}")
        return
    for idx in tqdm(range(n_images), desc=f"{topic_name}"):
        prompt = random.choice(prompt_list)
        try:
            img = pipe(prompt).images[0]
            img.save(os.path.join(out_dir, f"{topic_name}_{idx:04d}.jpg"))
        except Exception as e:
            print(f"Error generating image {idx} for {topic_name}: {e}")

# Define prompts for each topic
ai_topics = {
    "cars": ["a modern sedan on a city street, high realism", "a sports car parked outside", "a futuristic electric car"],
    "buildings": ["a glass skyscraper in sunlight", "a suburban house with a garden", "an old brick building downtown"],
    "medical": ["a hospital room with equipment", "a closeup of medical devices", "an x-ray machine in use"]
}

for topic, prompts in ai_topics.items():
    generate_topic(topic, prompts, f"data/ai/{topic}", n_images=7)
print("AI image generation complete.")

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 19 files:  32%|███▏      | 6/19 [01:53<04:05, 18.91s/it]

In [2]:
# 📌 Cell 4 — Stable Diffusion 1.5 Generator
# Loads SD 1.5 model and generates 50 images per topic.
from diffusers import StableDiffusionPipeline
pipe_sd15 = StableDiffusionPipeline.from_pretrained(
            'runwayml/stable-diffusion-v1-5',
            torch_dtype=torch.float32
        )
pipe_sd15 = pipe_sd15.to('cpu')

for topic, prompt in prompts.items():
            folder = f'data/ai/{topic}'
            for i in range(num_images):
                image = pipe_sd15(prompt).images[0]
                fname = f'sd15_{i:04d}.jpg'
                image.save(f'{folder}/{fname}')
                with open(f'{folder}/{fname}.json', 'w') as f:
                    json.dump({'prompt': prompt, 'model': 'SD15', 'index': i}, f)
        print('SD15 images generated.')

IndentationError: unindent does not match any outer indentation level (<string>, line 18)

In [ ]:
# 📌 Cell 5 — FLUX / Other Diffusion Model
# Loads Black Forest Labs FLUX or similar free model and generates 10 images per topic.
from diffusers import DiffusionPipeline
from tqdm import tqdm
import random

pipe_flux = DiffusionPipeline.from_pretrained(
    'blackforestai/flux-diffusion',
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32
)
pipe_flux = pipe_flux.to('cuda' if torch.cuda.is_available() else 'cpu')

num_flux = 10
for topic, prompt in prompts.items():
    folder = f'data/ai/{topic}'
    for i in tqdm(range(num_flux), desc=f'FLUX {topic}'):
        seed = random.randint(0, 999999)
        generator = torch.Generator(pipe_flux.device).manual_seed(seed)
        try:
            image = pipe_flux(prompt, generator=generator).images[0]
            fname = f'flux_{i:04d}.jpg'
            image.save(f'{folder}/{fname}')
            with open(f'{folder}/{fname}.json', 'w') as f:
                json.dump({'prompt': prompt, 'model': 'FLUX', 'index': i, 'seed': seed}, f)
        except Exception as e:
            print(f'Error generating {topic} #{i}:', e)
print('FLUX images generated.')

In [ ]:
# 📌 Cell 6 — DALL·E Showcase Scraper
import requests
from bs4 import BeautifulSoup
import os
import time
from urllib.parse import urljoin

dalle_url = 'https://openai.com/dall-e-3'
save_folder = 'data/ai/_extra/dalle_showcase'
os.makedirs(save_folder, exist_ok=True)
headers = {'User-Agent': 'Mozilla/5.0 (student project)'}
r = requests.get(dalle_url, headers=headers)
soup = BeautifulSoup(r.text, 'html.parser')
imgs = soup.find_all('img')
count = 0
for img in imgs:
    src = img.get('src')
    if not src:
        continue
    # Handle relative URLs
    if src.startswith('/'):
        src = urljoin(dalle_url, src)
    if not src.lower().endswith(('.jpg', '.jpeg', '.png')):
        continue
    try:
        img_data = requests.get(src, headers=headers)
        img_data.raise_for_status()
        fname = f'dalle_{count:04d}.jpg'
        with open(f'{save_folder}/{fname}', 'wb') as f:
            f.write(img_data.content)
        count += 1
    except Exception:
        continue
    time.sleep(0.5)
print(f'DALL·E showcase images downloaded: {count}')

In [ ]:
# 📌 Cell 7 — Midjourney Community Scraper
import requests
from bs4 import BeautifulSoup
import os
import time
from urllib.parse import urljoin

mj_url = 'https://www.midjourney.com/explore'
save_folder = 'data/ai/_extra/midjourney'
os.makedirs(save_folder, exist_ok=True)
headers = {'User-Agent': 'Mozilla/5.0 (student project)'}
r = requests.get(mj_url, headers=headers)
soup = BeautifulSoup(r.text, 'html.parser')
imgs = soup.find_all('img')
count = 0
for img in imgs:
    src = img.get('src')
    if not src:
        continue
    # Handle relative URLs
    if src.startswith('/'):
        src = urljoin(mj_url, src)
    if not src.lower().endswith(('.jpg', '.jpeg', '.png')):
        continue
    try:
        img_data = requests.get(src, headers=headers)
        img_data.raise_for_status()
        fname = f'mj_{count:04d}.jpg'
        with open(f'{save_folder}/{fname}', 'wb') as f:
            f.write(img_data.content)
        count += 1
    except Exception:
        continue
    time.sleep(0.5)
print(f'Midjourney images downloaded: {count}')

In [ ]:
# 📌 Cell 8 — Summary + Count
import glob

def count_images(folder):
    return len(glob.glob(f'{folder}/*.jpg')) + len(glob.glob(f'{folder}/*.jpeg')) + len(glob.glob(f'{folder}/*.png'))

cars = count_images('data/ai/cars')
buildings = count_images('data/ai/buildings')
medical = count_images('data/ai/medical')
dalle_showcase = count_images('data/ai/_extra/dalle_showcase')
midjourney = count_images('data/ai/_extra/midjourney')

total = cars + buildings + medical + dalle_showcase + midjourney

print('AI image summary:')
print(f'cars: {cars}')
print(f'buildings: {buildings}')
print(f'medical: {medical}')
print(f'dalle_showcase: {dalle_showcase}')
print(f'midjourney: {midjourney}')
print(f'Total: {total}')

The notebook was created. You can now run Section A1 to begin generating AI images.

In [ ]:
# Final validation cell: print counts and show sample images
import glob
import matplotlib.pyplot as plt
import os
from PIL import Image

def count_images(folder):
    return len(glob.glob(os.path.join(folder, '*.jpg')))

folders = [
    ("real/cars", "data/real/cars"),
    ("real/buildings", "data/real/buildings"),
    ("real/medical", "data/real/medical"),
    ("ai/cars", "data/ai/cars"),
    ("ai/buildings", "data/ai/buildings"),
    ("ai/medical", "data/ai/medical"),
]
for label, path in folders:
    print(f"{label:<15}: {count_images(path)}")

# Show 3 sample images from data/ai/cars
sample_imgs = glob.glob("data/ai/cars/*.jpg")[:3]
if sample_imgs:
    fig, axs = plt.subplots(1, len(sample_imgs), figsize=(12,4))
    for ax, img_path in zip(axs, sample_imgs):
        img = Image.open(img_path)
        ax.imshow(img)
        ax.axis('off')
        ax.set_title(os.path.basename(img_path))
    plt.suptitle("Sample AI-generated car images")
    plt.show()
else:
    print("No AI car images found to display.")